## Installing necessary libraries

In [187]:
%%bash

# Dependencies
pip install docx2txt
pip install strsimpy
pip install python-docx
pip install pandas

if ls docx2csv >/dev/null 2>&1; then
    echo "docx2csv exists."
else
    echo "Folder does not exist. Cloning docx2csv."
    git clone https://github.com/ivbeg/docx2csv.git
fi

docx2csv exists.


In [188]:
%%bash

source .env
cd docx2csv && echo "$PASSWORD" | sudo -S python3 setup.py install

running install
running bdist_egg
running egg_info
writing docx2csv.egg-info/PKG-INFO
writing dependency_links to docx2csv.egg-info/dependency_links.txt
writing entry points to docx2csv.egg-info/entry_points.txt
writing requirements to docx2csv.egg-info/requires.txt
writing top-level names to docx2csv.egg-info/top_level.txt
reading manifest file 'docx2csv.egg-info/SOURCES.txt'
writing manifest file 'docx2csv.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/docx2csv
copying build/lib/docx2csv/converter.py -> build/bdist.linux-x86_64/egg/docx2csv
copying build/lib/docx2csv/core.py -> build/bdist.linux-x86_64/egg/docx2csv
copying build/lib/docx2csv/__init__.py -> build/bdist.linux-x86_64/egg/docx2csv
copying build/lib/docx2csv/__main__.py -> build/bdist.linux-x86_64/egg/docx2csv
byte-compiling build/bdist.linux-x86_64/egg/docx2csv/converter.py to co

In [189]:
# ----- TEST DATA INPUT -----

# Computer Science Test Data.
# CURRENT_MAPPING="Lists_ComputerScience.docx"
# ORIGINAL_MAPPING="Original-Mapping-ComputerScience.csv"

# InformationSecurity Test Data.
CURRENT_MAPPING="Lists_MonashEngineering.docx"
ORIGINAL_MAPPING="Original-Mapping-MonashEngineering.csv"

In [190]:
# importing necessary libraries
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [191]:
# extract tables from word document
from docx2csv import extract_tables, extract
tables = extract_tables(CURRENT_MAPPING)

In [192]:
from docx import Document
document = Document(CURRENT_MAPPING)

In [193]:
def read_docx_table(document,table_num):
  table = document.tables[table_num-1]
  data = [[cell.text for cell in row.cells] for row in table.rows]
  df = pd.DataFrame(data)
  return df

## PLO TABLE

In [194]:
# Creating a dataframe for PLOs and it will accept 'n' number of PLOs
table_num=1
df = read_docx_table(document,table_num)
df.head(n=12)

,0,1
0,PO1,"Apply knowledge of mathematics, natural scienc..."
1,PO2,"Identify, formulate, survey research literatur..."
2,PO3,Design solutions for complex Software engineer...
3,PO4,Conduct investigations of complex Software eng...
4,PO5,"Create, select and apply appropriate technique..."
5,PO6,Apply reasoning informed by contextual knowled...
6,PO7,Understand and evaluate the sustainability and...
7,PO8,Apply ethical principles and commit to profess...
8,PO9,Communicate effectively on complex Software en...
9,PO10,"Function effectively as an individual, and as ..."


In [195]:
q1 = df.copy()
q1

,0,1
0,PO1,"Apply knowledge of mathematics, natural scienc..."
1,PO2,"Identify, formulate, survey research literatur..."
2,PO3,Design solutions for complex Software engineer...
3,PO4,Conduct investigations of complex Software eng...
4,PO5,"Create, select and apply appropriate technique..."
5,PO6,Apply reasoning informed by contextual knowled...
6,PO7,Understand and evaluate the sustainability and...
7,PO8,Apply ethical principles and commit to profess...
8,PO9,Communicate effectively on complex Software en...
9,PO10,"Function effectively as an individual, and as ..."


In [196]:
# assigning count vectorizer
count_vectorizer = CountVectorizer(stop_words='english', min_df=0.005)

In [197]:
# Remove integers

# Data preprocessing for PLO dataframe
q1[1] = q1[1].str.lower()
corpus = q1[1].tolist()
corpii = count_vectorizer.fit_transform(corpus)
corpus

['apply knowledge of mathematics, natural science, engineering fundamentals and specialization in software engineering to the solution of complex engineering problems;\n',
 'identify, formulate, survey research literature and analyze complex software engineering problems reaching substantiated conclusions using first principles of mathematics, natural sciences and engineering sciences;\n',
 'design solutions for complex software engineering problems and design systems, components or processes that meet specified needs;',
 'conduct investigations of complex software engineering problems using research-based knowledge and research methods including design of experiments, analysis and interpretation of data, and synthesis of information to provide valid conclusions;',
 'create, select and apply appropriate techniques, resources, and modern engineering and it tools, including prediction and modelling, to complex software engineering problems, with an understanding of the limitations;',
 'a

In [198]:
corpii

<12x128 sparse matrix of type '<class 'numpy.int64'>'
	with 182 stored elements in Compressed Sparse Row format>

In [199]:
# extracting features names from PLO table
feature_names = count_vectorizer.get_feature_names_out()
feature_names

array(['ability', 'able', 'activities', 'analysis', 'analyze', 'apply',
       'appropriate', 'assess', 'based', 'broadest', 'change', 'clear',
       'commit', 'communicate', 'community', 'complex', 'components',
       'comprehend', 'conclusions', 'conduct', 'consequent', 'context',
       'contexts', 'contextual', 'create', 'cultural', 'data', 'decision',
       'demonstrate', 'design', 'disciplinary', 'diverse',
       'documentation', 'economic', 'effective', 'effectively', 'engage',
       'engineering', 'environmental', 'ethical', 'ethics', 'evaluate',
       'experiments', 'formulate', 'function', 'fundamentals', 'health',
       'identify', 'impact', 'including', 'independent', 'individual',
       'information', 'informed', 'instructions', 'interpretation',
       'investigations', 'issues', 'knowledge', 'large', 'leader',
       'learning', 'legal', 'life', 'limitations', 'literature', 'long',
       'make', 'making', 'manage', 'management', 'mathematics', 'meet',
       'me

In [200]:
len(feature_names)

128

In [201]:
# Converting features to vector form and create a dataframe
X1 = pd.DataFrame(corpii.toarray(), columns=feature_names)

## CLO TABLE

In [202]:
# Creating a dataframe for CLOs and it will accept 'n' number of CLOs
table_num=2
df1 = read_docx_table(document,table_num)
p1 = df1.copy()
p1

,0,1
0,LO1-MA,Describe the underlying theoretical basis of t...
1,LO2-MA,Design a relational database model based on th...
2,LO3-MA,Implement a database based on a sound database...
3,LO4-MA,Contrast the differences between non-relationa...
4,LO5-MA,Develop programming structures within a databa...
...,...,...
82,LO2-MA,Evaluate and apply the basic tools and techniq...
83,LO3-MA,"Evaluate the interconnection of quality, heath..."
84,LO4-MA,Understand and evaluate the need for sustainab...
85,LO5-MA,Explain the contract laws and apply ethical pr...


In [203]:
# Data preprocessing for CLO dataframe
p1[1] = p1[1].str.lower()
corpus11 = p1[1].tolist()
corpii11 = count_vectorizer.fit_transform(corpus11)

In [204]:
# extracting features names from CLO table
feature_names1 = count_vectorizer.get_feature_names_out()

In [205]:
len(feature_names1)

454

In [206]:
# Converting features to vector form and create a dataframe
X2 = pd.DataFrame(corpii11.toarray(), columns=feature_names1)
X2

,ability,abstract,acceptable,accordance,achieve,achieved,actions,activities,adopt,advanced,...,visualisations,vocabulary,wired,wireless,work,working,worst,write,writing,written
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
83,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
84,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
85,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [207]:
X2.head()

,ability,abstract,acceptable,accordance,achieve,achieved,actions,activities,adopt,advanced,...,visualisations,vocabulary,wired,wireless,work,working,worst,write,writing,written
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [208]:
# adding column index to the CLO table
U2 = pd.concat([df1[0], X2], axis=1)
U2.set_index(0, inplace=True)

In [209]:
U2.head()

,ability,abstract,acceptable,accordance,achieve,achieved,actions,activities,adopt,advanced,...,visualisations,vocabulary,wired,wireless,work,working,worst,write,writing,written
0,,,,,,,,,,,,,,,,,,,,,
LO1-MA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
LO2-MA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
LO3-MA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
LO4-MA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
LO5-MA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [210]:
# adding column index to the PLO table
U1 = pd.concat([df[0], X1], axis=1)
U1.set_index(0, inplace=True)

In [211]:
U1

,ability,able,activities,analysis,analyze,apply,appropriate,assess,based,broadest,...,teams,techniques,technological,tools,understand,understanding,using,valid,work,write
0,,,,,,,,,,,,,,,,,,,,,
PO1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PO2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
PO3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PO4,0,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,1,1,0,0
PO5,0,0,0,0,0,1,1,0,0,0,...,0,1,0,1,0,1,0,0,0,0
PO6,0,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
PO7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
PO8,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PO9,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


## Intersection method for both CLOs and PLOs

### Generalised list of words

In [212]:
append_words = list(map(str.lower,['Cite', 'Define', 'Describe', 'Draw', 'Enumerate', 'Identify' 'Index', 'Indicate', 'Label', 'List', 'Match', 'Meet', 'Name', 'Outline', 'Point', 'Quote', 'Read', 'Recall', 'Recite', 'Recognize', 'Record', 'Repeat', 'Reproduce','Review',
'Select', 'State', 'Study', 'Tabulate', 'Trace', 'Write', 'Add', 'Approximate', 'Articulate', 'Associate', 'Characterize', 'Clarify', 'Classify', 'Compare', 'Compute', 'Contrast', 'Convert', 'Defend', 'Detail', 'Differentiate',
'Discuss', 'Distinguish', 'Elaborate', 'Estimate', 'Example', 'Explain', 'Express', 'Extend', 'Extrapolate', 'Factor', 'Generalize', 'Give', 'Infer', 'Interact', 'Interpolate', 'Interpret', 'Observe', 'Paraphrase', 'Picture graphically',
'Predict', 'Rewrite', 'Subtract', 'Summarize', 'Translate', 'Visualize', 'Acquire', 'Adapt', 'Allocate', 'Alphabetize', 'Apply', 'Ascertain', 'Assign', 'Attain', 'Avoid', 'Back up', 'Calculate', 'Capture', 'Change', 'Complete', 'Construct', 
'Customize', 'Demonstrate', 'Depreciate', 'Derive', 'Determine', 'Diminish', 'Discover', 'Employ', 'Examine', 'Exercise', 'Explore', 'Expose', 'Figure', 'Graph', 'Handle', 'Illustrate', 'Interconvert', 'Investigate', 'Manipulate', 'Modify', 
'Operate', 'Personalize', 'Plot','Practice', 'Prepare', 'Price', 'Process', 'Produce', 'Project', 'Provide', 'Relate', 'Round off', 'Sequence', 'Show', 'Simulate', 'Sketch', 'Solve', 'Subscribe', 'Transcribe', 'Use', 'Analyze', 'Audit', 
'Blueprint', 'Breadboard', 'Break down', 'Confirm', 'Correlate', 'Detect', 'Diagnose', 'Diagram', 'Discriminate', 'Dissect', 'Document', 'Ensure', 'Figure out', 'File', 'Group', 'Interrupt', 'Inventory', 'Layout', 'Manage', 'Maximize', 
'Minimize', 'Optimize', 'Order', 'Point out', 'Prioritize', 'Proofread', 'Query', 'Separate', 'Subdivide', 'Train', 'Transform', 'Appraise', 'Assess', 'Conclude', 'Counsel', 'Criticize', 'Critique', 'Evaluate', 'Grade', 'Hire', 'Judge', 
'Justify', 'Measure', 'Prescribe', 'Rank', 'Rate', 'Recommend', 'Release', 'Support', 'Test', 'Validate', 'Verify', 'Abstract', 'Animate', 'Arrange', 'Assemble', 'Budget', 'Categorize', 'Code', 'Combine', 'Compile', 'Compose', 'Cope', 
'Correspond', 'Create', 'Cultivate', 'Debug', 'Depict', 'Design', 'Develop', 'Devise', 'Dictate', 'Enhance', 'Facilitate', 'Format', 'Formulate', 'Generate', 'Import', 'Improve', 'Incorporate', 'Integrate', 'Interface', 'Join', 'Lecture', 
'Model', 'Network', 'Organize', 'Overhaul', 'Plan', 'Portray', 'Program', 'Rearrange', 'Reconstruct', 'Reorganize', 'Revise', 'Specify']))

In [213]:
# using + operator to concat the generalised list of words to the PLO list
train_column = list(feature_names) + append_words

In [214]:
# CLO list of words
test_column = feature_names1
test_column

array(['ability', 'abstract', 'acceptable', 'accordance', 'achieve',
       'achieved', 'actions', 'activities', 'adopt', 'advanced', 'agreed',
       'algebra', 'algorithmic', 'algorithms', 'allocate', 'analyse',
       'analysis', 'applications', 'apply', 'approach', 'approaches',
       'appropriate', 'appropriately', 'architectural', 'architectures',
       'artifacts', 'aspects', 'assembly', 'assess', 'assumptions',
       'assurance', 'attributes', 'australia', 'automated',
       'availability', 'available', 'backend', 'balances', 'based',
       'basic', 'basis', 'beam', 'behaviour', 'behaviours', 'best', 'big',
       'board', 'boundaries', 'brief', 'business', 'c3', 'c4', 'c5', 'c6',
       'calculus', 'capability', 'carry', 'case', 'chemical', 'choose',
       'circuit', 'circuits', 'civil', 'classes', 'client', 'clients',
       'code', 'codes', 'collaboration', 'collection', 'combination',
       'commit', 'commitment', 'common', 'communicate', 'communicated',
       'comm

In [215]:
# Intersection method for extracting common column names from the tables (both CLO AND PLO)
# comparing whether the CLO column name is present in the PLO column names or not
train_column = list(feature_names) + append_words # (PLO table ) (# using + operator to concat PLO words and list of generalized words)
test_column = list(feature_names1)   # (CLO table)

In [216]:
# This is the column names from both the tables (using intersection)
common_column = list(set(train_column).intersection(set(test_column)))
common_column

['limitations',
 'contrast',
 'evaluate',
 'allocate',
 'societal',
 'norms',
 'components',
 'verify',
 'ability',
 'documentation',
 'health',
 'plan',
 'write',
 'support',
 'cultural',
 'safety',
 'leader',
 'understanding',
 'demonstrate',
 'environmental',
 'data',
 'assess',
 'natural',
 'knowledge',
 'consequent',
 'document',
 'including',
 'project',
 'community',
 'modern',
 'conduct',
 'need',
 'determine',
 'legal',
 'resources',
 'sciences',
 'responsibilities',
 'tools',
 'model',
 'issues',
 'techniques',
 'sustainability',
 'use',
 'complex',
 'solution',
 'code',
 'appropriate',
 'context',
 'solutions',
 'discuss',
 'based',
 'impact',
 'develop',
 'apply',
 'understand',
 'professional',
 'manage',
 'ensure',
 'debug',
 'explain',
 'using',
 'recognize',
 'investigations',
 'network',
 'compare',
 'problems',
 'multi',
 'needs',
 'activities',
 'effectively',
 'analysis',
 'ethics',
 'software',
 'justify',
 'ethical',
 'practice',
 'systems',
 'process',
 'devise',

In [217]:
print(common_column)

['limitations', 'contrast', 'evaluate', 'allocate', 'societal', 'norms', 'components', 'verify', 'ability', 'documentation', 'health', 'plan', 'write', 'support', 'cultural', 'safety', 'leader', 'understanding', 'demonstrate', 'environmental', 'data', 'assess', 'natural', 'knowledge', 'consequent', 'document', 'including', 'project', 'community', 'modern', 'conduct', 'need', 'determine', 'legal', 'resources', 'sciences', 'responsibilities', 'tools', 'model', 'issues', 'techniques', 'sustainability', 'use', 'complex', 'solution', 'code', 'appropriate', 'context', 'solutions', 'discuss', 'based', 'impact', 'develop', 'apply', 'understand', 'professional', 'manage', 'ensure', 'debug', 'explain', 'using', 'recognize', 'investigations', 'network', 'compare', 'problems', 'multi', 'needs', 'activities', 'effectively', 'analysis', 'ethics', 'software', 'justify', 'ethical', 'practice', 'systems', 'process', 'devise', 'life', 'investigate', 'member', 'mathematics', 'relevant', 'abstract', 'revi

In [218]:
len(common_column)

105

In [219]:
# Filter the common column values from the CLO table
U3 = U2.filter(list(common_column), axis=1)

In [220]:
U3.head()

,limitations,contrast,evaluate,allocate,societal,norms,components,verify,ability,documentation,...,diverse,identify,meet,select,fundamentals,state,work,principles,validate,engineering
0,,,,,,,,,,,,,,,,,,,,,
LO1-MA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
LO2-MA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
LO3-MA,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
LO4-MA,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
LO5-MA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [221]:
# extracting first row from PLO table and make a dataframe
Cs = []
for x in range(len(df)):
    Cs.append(U1.loc[['PO'+str(x+1)]])

In [222]:
# Concatenating these extracted each PLOs with 'n' number of CLOs
Dds = []
for x in range(len(df)):
    Dds.append(pd.concat([Cs[x],U3], sort=True))

In [223]:
# Filling the nan values of the concatenated dataframes
Ds = []
for x in range(len(df)):
    Ds.append(Dds[x].fillna(0))

## Calculate Cosine similarity

In [224]:
# Calculate cosine similarity for concatenated dataframes and create a new dataframe
for x in range(len(df)):
    Dds[x] = pd.DataFrame(cosine_similarity(Ds[x], dense_output=True))
Ds

[        ability  able  abstract  activities  allocate  analysis  analyze  \
 0                                                                          
 PO1           0   0.0       0.0           0       0.0         0      0.0   
 LO1-MA        0   0.0       0.0           0       0.0         0      0.0   
 LO2-MA        0   0.0       0.0           0       0.0         0      0.0   
 LO3-MA        0   0.0       0.0           0       0.0         0      0.0   
 LO4-MA        0   0.0       0.0           0       0.0         0      0.0   
 ...         ...   ...       ...         ...       ...       ...      ...   
 LO2-MA        0   0.0       0.0           0       0.0         0      0.0   
 LO3-MA        0   0.0       0.0           0       0.0         0      0.0   
 LO4-MA        0   0.0       0.0           0       0.0         0      0.0   
 LO5-MA        0   0.0       0.0           0       0.0         0      0.0   
 LO6-MA        0   0.0       0.0           1       0.0         0      0.0   

In [225]:
# Extract the '0'th column because it has the CLO-PLO  cosine similarity values. We are neglecting the remaining ones.
# Renaming the '0'th column name to 'Pn' ['P1, P2, P3, P4, ... 'Pn']
for x in range(len(df)):
    Dds[x].rename(columns = {0 :'PO'+str(x+1)}, inplace = True)

Dds

[         PO1         1         2         3         4    5    6         7  \
 0   1.000000  0.000000  0.000000  0.000000  0.000000  0.0  0.0  0.372678   
 1   0.000000  1.000000  0.577350  0.000000  0.707107  0.0  0.0  0.000000   
 2   0.000000  0.577350  1.000000  0.666667  0.408248  0.0  0.0  0.000000   
 3   0.000000  0.000000  0.666667  1.000000  0.000000  0.0  0.0  0.000000   
 4   0.000000  0.707107  0.408248  0.000000  1.000000  0.0  0.0  0.000000   
 ..       ...       ...       ...       ...       ...  ...  ...       ...   
 83  0.070711  0.000000  0.000000  0.000000  0.000000  0.0  0.0  0.000000   
 84  0.212132  0.000000  0.000000  0.000000  0.000000  0.0  0.0  0.105409   
 85  0.461880  0.000000  0.000000  0.000000  0.000000  0.0  0.0  0.258199   
 86  0.269680  0.000000  0.000000  0.000000  0.000000  0.0  0.0  0.100504   
 87  0.521749  0.000000  0.000000  0.000000  0.000000  0.0  0.0  0.222222   
 
            8         9  ...        78        79        80        81      

In [226]:
# Concatenating each  '0'th column from different cosine similarity dataframes
Ddn = []
for x in range(len(df)):
    Ddn.append(Dds[x]['PO'+str(x+1)])

d = pd.concat(Ddn, axis=1)
d

,PO1,PO2,PO3,PO4,PO5,PO6,PO7,PO8,PO9,PO10,PO11,PO12
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.298142,0.230940,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.447214,0.230940,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
83,0.070711,0.064550,0.081650,0.000000,0.212132,0.063246,0.079057,0.100000,0.000000,0.000000,0.000000,0.182574
84,0.212132,0.129099,0.081650,0.063246,0.141421,0.569210,0.316228,0.400000,0.121716,0.000000,0.000000,0.182574
85,0.461880,0.263523,0.200000,0.154919,0.288675,0.309839,0.839146,0.244949,0.198762,0.000000,0.143223,0.149071
86,0.269680,0.184637,0.077850,0.060302,0.202260,0.361814,0.226134,0.953463,0.116052,0.000000,0.000000,0.261116


In [227]:
# '0'th column gives us 1 which means each PLO map with own PLO.
# So we are removing that column.
dd = d[1:]
dd

,PO1,PO2,PO3,PO4,PO5,PO6,PO7,PO8,PO9,PO10,PO11,PO12
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.298142,0.230940,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.447214,0.230940,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
83,0.070711,0.064550,0.081650,0.000000,0.212132,0.063246,0.079057,0.100000,0.000000,0.000000,0.000000,0.182574
84,0.212132,0.129099,0.081650,0.063246,0.141421,0.569210,0.316228,0.400000,0.121716,0.000000,0.000000,0.182574
85,0.461880,0.263523,0.200000,0.154919,0.288675,0.309839,0.839146,0.244949,0.198762,0.000000,0.143223,0.149071
86,0.269680,0.184637,0.077850,0.060302,0.202260,0.361814,0.226134,0.953463,0.116052,0.000000,0.000000,0.261116


In [228]:
# resetting index
dd.reset_index(inplace = True)
dd.drop(['index'], axis=1, inplace = True)
dd

/tmp/ipykernel_29680/1086022003.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dd.drop(['index'], axis=1, inplace = True)


,PO1,PO2,PO3,PO4,PO5,PO6,PO7,PO8,PO9,PO10,PO11,PO12
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.298142,0.230940,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.447214,0.230940,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
82,0.070711,0.064550,0.081650,0.000000,0.212132,0.063246,0.079057,0.100000,0.000000,0.000000,0.000000,0.182574
83,0.212132,0.129099,0.081650,0.063246,0.141421,0.569210,0.316228,0.400000,0.121716,0.000000,0.000000,0.182574
84,0.461880,0.263523,0.200000,0.154919,0.288675,0.309839,0.839146,0.244949,0.198762,0.000000,0.143223,0.149071
85,0.269680,0.184637,0.077850,0.060302,0.202260,0.361814,0.226134,0.953463,0.116052,0.000000,0.000000,0.261116


In [229]:
# store the matrix into csv file
dd.to_csv('pseudocodematrix.csv', index=False)

In [230]:
# print the matrix
print(dd)

         PO1       PO2       PO3       PO4       PO5       PO6       PO7  \
0   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
1   0.000000  0.000000  0.298142  0.230940  0.000000  0.000000  0.000000   
2   0.000000  0.000000  0.447214  0.230940  0.000000  0.000000  0.000000   
3   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
4   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
..       ...       ...       ...       ...       ...       ...       ...   
82  0.070711  0.064550  0.081650  0.000000  0.212132  0.063246  0.079057   
83  0.212132  0.129099  0.081650  0.063246  0.141421  0.569210  0.316228   
84  0.461880  0.263523  0.200000  0.154919  0.288675  0.309839  0.839146   
85  0.269680  0.184637  0.077850  0.060302  0.202260  0.361814  0.226134   
86  0.521749  0.340207  0.258199  0.200000  0.372678  0.333333  0.416667   

         PO8       PO9      PO10      PO11      PO12  
0   0.000000  0.000000  0.000000

In [231]:
## This code loads the vector file into the word_vectors variable
## Download the vector file from https://fasttext.cc/docs/en/english-vectors.html (first file on the website), unzip the file and store in your local development folder
## Note: This piece of code may take upto an hour or two to run depending on your pc specs.
## My i5 8th gen with 8gig ram took 58mins to run.

# from gensim.models import KeyedVectors

# # Path to the downloaded .vec file
# path_to_vectors = 'wiki-news-300d-1M.vec'
# # path_to_vectors = 'wiki.en.vec'
# # Load the word vectors
# word_vectors = KeyedVectors.load_word2vec_format(path_to_vectors)

# # Find similar words
# similar_words = word_vectors.most_similar('cat')

# # Calculate word similarity
# similarity = word_vectors.similarity('cat', 'dog')

# # Perform vector arithmetic
# result = word_vectors['king'] - word_vectors['man'] + word_vectors['woman']


In [232]:
import pandas as pd
import spacy

file_path = "Bloom and SOLO Verbs.xlsx"

xls = pd.ExcelFile(file_path)
sheet_names = xls.sheet_names
# print(sheet_names)

sources = {} # Copy verbs from excel into var to maniupulate easier
verbs = {   # Complete list of verbs sorted into their taxonomy. Includes # of occurrences and the potential sources and levels the verb could be mapped to
    "Cognitive": {},
    "Affective": {},
    "Psychomotor": {},
    "SOLO": {}
}

domain_templates = {    # Template structure for each taxonomy
    "Cognitive": {
        "Remembering": set(),
        "Understanding": set(),
        "Applying": set(),
        "Analysing": set(),
        "Evaluating": set(),
        "Creating": set()
    },
    "Affective": {
        "Receiving": set(),
        "Responding": set(),
        "Valuing": set(),
        "Organisation": set(),
        "Characterisation": set(),
    },
    "Psychomotor": {
        "Perception": set(),
        "Set": set(),
        "Guided Response": set(),
        "Mechanism": set(),
        "Complex Overt Response": set(),
        "Adaptation": set(),
        "Origination": set()    
    },
    "SOLO": {
        "Prestructural": set(),
        "Unistructural": set(),
        "Multistructural": set(),
        "Relational": set(),
        "Extended Abstract": set()
    }
}

# Var to store final mappings of verbs for each taxonomy
mapped_verbs = domain_templates

# Read all sources and store count of verbs into var
for domain_key, domain_item in mapped_verbs.items():    # For each taxonomy
    for current_sheet_name in sheet_names:      # For each source sheet
        if domain_key in current_sheet_name:    # If the current taxonomy matches the current source sheet
            current_sheet = pd.read_excel(xls, current_sheet_name)  # Read the sheet into a var
            sources[current_sheet_name] = domain_templates.get(domain_key)  # Add the appropriate taxonomy template structure
            for j in range(len(domain_item.keys())):        # For each level of the taxonomy
                current_column = current_sheet.columns[j]
                column_values = current_sheet[current_column].values.tolist()
                for verb in range(len(column_values)):      # For each verb in the level
                    if type(column_values[verb])!= str or len(str(column_values[verb]).strip()) == 0:   # Validate the verb is a string
                        continue
                    v = str(column_values[verb]).strip().lower()    # Format verb
                    sources[current_sheet_name][current_column].add(v)
                    if v not in verbs:      # If this is the first instance of the verb for the taxonomy init the dict
                        verbs[domain_key][v] = {
                            "count": 1,
                            "potentials": [{
                                "level": current_column, 
                                "source": current_sheet_name
                                }]
                        }
                    else:   # Else add vars
                        verbs[domain_key][v]["count"] += 1
                        verbs[domain_key][v]["potentials"].append({
                                "level": current_column, 
                                "source": current_sheet_name
                                })

# print(sources)
# print(verbs)
# test = verbs
# solo = verbs["SOLO"]
# # print(solo)
# for verb in solo.items():
#     print(verb)


# Classify each verb in a taxonomy into a single level
for domain_key, domain_item in verbs.items():   # For each taxonomy
    for v in domain_item:   # For each verb
        if verbs[domain_key][v]["count"] == 1:  # If there is only one instance of the verb, map it
            mapped_verbs[domain_key][verbs[domain_key][v]["potentials"][0]["level"]].add(v)
        else:
            chosen_source = None
            chosen_level = None
            potential_sources = []
            potential_levels = []
            for l in verbs[domain_key][v]["potentials"]:    # Iterate through the potential level mappings
                if l["source"] not in potential_sources or l["level"] not in potential_levels:
                    potential_sources.append(l["source"])
                    potential_levels.append(l["level"])
            for sheet in sheet_names:   # For each source, map verb to first matching source (Assumes sources are sorted in order of priority for mapping)
                if sheet in potential_sources:
                    chosen_source = sheet
                    chosen_level = potential_levels[potential_sources.index(sheet)]
                    break
            mapped_verbs[domain_key][chosen_level].add(v)

## Function to identify verbs in a sentence
def identify_verbs(sentence):
    # Load the English language model in spaCy
    nlp = spacy.load('en_core_web_sm')
    
    # Process the sentence using spaCy
    doc = nlp(sentence)
    
    # Extract the verbs from the processed sentence
    verbs = [token.lemma_ for token in doc if token.pos_ == 'VERB']
    
    return verbs

In [233]:
def bloom_mapping(sentences):

    mappings = []

    final_level = None
    passed_mappings = 0
    failed_mappings = 0
    total_mappings = len(sentences)
    for i in range(len(sentences)): # Iterates over the LOs
        identified_verbs = identify_verbs(sentences[i])

        score_list = {
            "Cognitive": {
                "Remembering": 0,
                "Understanding": 0,
                "Applying": 0,
                "Analysing": 0,
                "Evaluating": 0,
                "Creating": 0
            },
            "Affective": {
                "Receiving": 0,
                "Responding": 0,
                "Valuing": 0,
                "Organisation": 0,
                "Characterisation": 0
            },
            "Psychomotor": {
                "Perception": 0,
                "Set": 0,
                "Guided Response": 0,
                "Mechanism": 0,
                "Complex Overt Response": 0,
                "Adaptation": 0,
                "Origination": 0    
            },
            "SOLO": {
                "Prestructural": 0,
                "Unistructural": 0,
                "Multistructural": 0,
                "Relational": 0,
                "Extended Abstract": 0
            }
        }
        for taxonomy_key, taxonomy_item in mapped_verbs.items():
            for level in taxonomy_item.keys(): # Level
                for verb in mapped_verbs[taxonomy_key][level]: # Verb
                    similarity_score = 0
                    for l in range(len(identified_verbs)):
                        try:    # Currently some of the 'verbs' identified are phrases rather than words and it was throwing errors so this is a temp solution 
                            similarity_score += word_vectors.similarity(identified_verbs[l], verb)
                        except:
                            pass
                    score_list[taxonomy_key][level] += similarity_score

        # Identify level based on similarity
        max_score = {
            "Cognitive": { "Level": None, "Score": 0 },
            "Affective": { "Level": None, "Score": 0 },
            "Psychomotor": { "Level": None, "Score": 0 },
            "SOLO": { "Level": None, "Score": 0 }
        }
        for t_key, t_item in score_list.items():
            for l in t_item:
                if max_score[t_key]["Score"] < score_list[t_key][l]:
                    max_score[t_key] = { "Level": l, "Score": score_list[t_key][l] }
        mappings.append(max_score)

    return mappings

# TODO: Classify the verbs in each of the learning outcomes
ulo_sentences = p1[1].to_list()
plo_sentences = df[1].to_list()

ulo_classifications = bloom_mapping(ulo_sentences)
plo_classifications = bloom_mapping(plo_sentences)

# Iteration 1: Assume both CLOs and POs are classifiable into Cognitive, Affective, Psychomotor Levels. 
# If levels match, boost the CLO to PO coefficient by 0.1
# If levels different, don't boost

In [234]:
# for classification in ulo_classifications:
#     print(classification)
for x in range(len(df)):
    for i in range(len(ulo_sentences)):
        if ulo_classifications[i]['Cognitive']['Level'] == plo_classifications[x]['Cognitive']['Level']:
            dd['PO'+str(x+1)][i] += 0.2 # Add Offset
        else:
            dd['PO'+str(x+1)][i] -= 0.2 # Add Offset

/tmp/ipykernel_29680/1037742226.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dd['PO'+str(x+1)][i] += 0.2 # Add Offset
/tmp/ipykernel_29680/1037742226.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dd['PO'+str(x+1)][i] -= 0.2 # Add Offset


## Setting threshold value (taking min and max of each column and divided by 2)
## threshold value = (min +max)/2

In [235]:

# TODO: Alter the threshold based on matching hierarchy type & bloom verb instead of simply using (column_max+column_min)/2

# Setting threshold value 
# Taking min max average of each column and set that as a threshold value

# This will change the coefficients into 0 or 1 mappings in the dd dataframe
for x in range(len(df)):
    tes = dd['PO'+str(x+1)].values.min()
    tes1 = dd['PO'+str(x+1)].values.max()
    tt1 = (tes+tes1)/2
    
    if tt1 == 0:
      dd['PO'+str(x+1)] = dd['PO'+str(x+1)] 
    else:
      dd['PO'+str(x+1)] = dd['PO'+str(x+1)].apply(lambda x: 1 if x >= tt1 else 0)
# dd

/tmp/ipykernel_29680/218065778.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dd['PO'+str(x+1)] = dd['PO'+str(x+1)].apply(lambda x: 1 if x >= tt1 else 0)


In [236]:
# dd

dd.to_csv('PLO-CLOmapping.csv', index=False)

In [237]:
dd.head()

,PO1,PO2,PO3,PO4,PO5,PO6,PO7,PO8,PO9,PO10,PO11,PO12
0,0,0,0,1,0,0,0,0,0,0,1,0
1,0,0,1,1,0,0,0,0,1,0,1,0
2,0,0,1,1,0,0,0,0,1,0,1,0
3,0,0,0,1,0,0,0,0,0,0,1,0
4,0,0,0,1,0,0,0,0,0,0,1,0


In [238]:
# human generated output
d= pd.read_csv(ORIGINAL_MAPPING)
d.head()

,PO1,PO2,PO3,PO4,PO5,PO6,PO7,PO8,PO9,PO10,PO11,PO12
0,1,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,0,0


In [239]:
df3 = d.copy()

In [240]:
dd

,PO1,PO2,PO3,PO4,PO5,PO6,PO7,PO8,PO9,PO10,PO11,PO12
0,0,0,0,1,0,0,0,0,0,0,1,0
1,0,0,1,1,0,0,0,0,1,0,1,0
2,0,0,1,1,0,0,0,0,1,0,1,0
3,0,0,0,1,0,0,0,0,0,0,1,0
4,0,0,0,1,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
82,0,0,0,1,1,0,0,0,0,0,1,1
83,1,1,0,1,1,1,1,1,1,0,1,1
84,1,1,1,1,1,1,1,0,1,0,1,1
85,1,1,0,1,1,1,1,1,1,0,1,1


In [241]:
df3

,PO1,PO2,PO3,PO4,PO5,PO6,PO7,PO8,PO9,PO10,PO11,PO12
0,1,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
82,0,0,0,0,0,0,0,0,0,0,0,1
83,0,0,0,0,0,1,0,0,0,0,0,0
84,0,0,0,0,0,0,1,0,0,0,0,0
85,0,0,0,0,0,0,0,1,0,0,0,0


In [242]:
for x in range(len(df)):
  df3['PO'+str(x+1)] = np.where(dd['PO'+str(x+1)] == df3['PO'+str(x+1)], 'True', 'False')

In [243]:
df3.head()

,PO1,PO2,PO3,PO4,PO5,PO6,PO7,PO8,PO9,PO10,PO11,PO12
0,False,True,True,False,True,True,True,True,True,True,False,True
1,True,False,False,False,True,True,True,True,False,True,False,True
2,True,True,True,False,True,True,True,True,False,True,False,True
3,False,True,True,False,True,True,True,True,True,True,False,True
4,True,True,False,False,True,True,True,True,True,True,False,True


In [244]:
for x in range(len(df)):
  df3['PO'+str(x+1)] = df3['PO'+str(x+1)].replace('True', 1)
  df3['PO'+str(x+1)] = df3['PO'+str(x+1)].replace('False', 0)

In [245]:
df3.head()

,PO1,PO2,PO3,PO4,PO5,PO6,PO7,PO8,PO9,PO10,PO11,PO12
0,0,1,1,0,1,1,1,1,1,1,0,1
1,1,0,0,0,1,1,1,1,0,1,0,1
2,1,1,1,0,1,1,1,1,0,1,0,1
3,0,1,1,0,1,1,1,1,1,1,0,1
4,1,1,0,0,1,1,1,1,1,1,0,1


In [246]:
# calculating accuracy of the table
df3['acc'] = df3.mean(axis=1)
df3.head(n=100)

,PO1,PO2,PO3,PO4,PO5,PO6,PO7,PO8,PO9,PO10,PO11,PO12,acc
0,0,1,1,0,1,1,1,1,1,1,0,1,0.750000
1,1,0,0,0,1,1,1,1,0,1,0,1,0.583333
2,1,1,1,0,1,1,1,1,0,1,0,1,0.750000
3,0,1,1,0,1,1,1,1,1,1,0,1,0.750000
4,1,1,0,0,1,1,1,1,1,1,0,1,0.750000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,1,1,1,0,0,1,1,1,1,1,0,1,0.750000
83,0,0,1,0,0,1,0,0,0,1,0,0,0.250000
84,0,0,0,0,0,0,1,1,0,1,0,0,0.250000
85,0,0,1,0,0,0,0,1,0,1,0,0,0.250000


In [247]:
df4 = pd.concat([df1[0], df3], axis=1)
df4.head(n=100)

,0,PO1,PO2,PO3,PO4,PO5,PO6,PO7,PO8,PO9,PO10,PO11,PO12,acc
0,LO1-MA,0,1,1,0,1,1,1,1,1,1,0,1,0.750000
1,LO2-MA,1,0,0,0,1,1,1,1,0,1,0,1,0.583333
2,LO3-MA,1,1,1,0,1,1,1,1,0,1,0,1,0.750000
3,LO4-MA,0,1,1,0,1,1,1,1,1,1,0,1,0.750000
4,LO5-MA,1,1,0,0,1,1,1,1,1,1,0,1,0.750000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,LO2-MA,1,1,1,0,0,1,1,1,1,1,0,1,0.750000
83,LO3-MA,0,0,1,0,0,1,0,0,0,1,0,0,0.250000
84,LO4-MA,0,0,0,0,0,0,1,1,0,1,0,0,0.250000
85,LO5-MA,0,0,1,0,0,0,0,1,0,1,0,0,0.250000


In [248]:
df4.set_index(0, inplace=True)
df4.head(n=100)

df4.to_csv('WasMappingSuccessful.csv')

In [249]:
df4['acc'].mean()

0.5440613026819923